In [1]:
import nltk
#from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import torch
from tqdm import tqdm

# Load the model and tokenizer
# model_name = 'Helsinki-NLP/opus-mt-tc-big-en-pt'
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to('cuda')  # Use GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt").to(device)
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer.src_lang = "en_XX"

def translate_text(text):
    # Split the text into sentences
    sentences = nltk.sent_tokenize(text)
    
    # Create batches of sentences
    batch_size = 16  # Increase this as needed
    batches = [sentences[i:i+batch_size] for i in range(0, len(sentences), batch_size)]
    progress_bar = tqdm(total=len(batches), desc="Translating")

    translated_text = ""
    for batch in batches:
        # Tokenize the batch
        inputs = tokenizer(batch, return_tensors='pt', padding=True, truncation=True, max_length=512).to('cuda')
        
        # Generate translations
        translations = model.generate(**inputs)
        
        # Decode the translations
        translated_batch = [tokenizer.decode(t, skip_special_tokens=True) for t in translations]
        
        # Add the translated batch to the overall translation
        translated_text += " ".join(translated_batch)
        progress_bar.update(1)
    progress_bar.close()
    return translated_text



def translate_and_output_sentences(text):
    # Split the text into sentences
    sentences = nltk.sent_tokenize(text)
    batch_size = 16  # Increase this as needed
    batches = [sentences[i:i+batch_size] for i in range(0, len(sentences), batch_size)]
    progress_bar = tqdm(total=len(batches), desc="Translating")

    # Prepare the output
    output = []
    translated_text = ""

    for batch in batches:
        # Tokenize the sentence
        inputs = tokenizer(batch, return_tensors='pt', padding=True, truncation=True, max_length=512).to('cuda')
        
        # Generate translation
        translations = model.generate(**inputs, forced_bos_token_id=tokenizer.lang_code_to_id["pt_XX"])
        
        # Decode the translation
        translated_batch = [tokenizer.decode(t, skip_special_tokens=True) for t in translations]
        translated_text += " ".join(translated_batch)

        # Append original and translated sentences to the output
        progress_bar.update(1)

        for original, translated in zip(batch, translated_batch):
            output.append(f"Original: {original}\nTranslated: {translated}\n")
    progress_bar.close()
    return output, translated_text




# Read the book text

input_file = r"C:\Users\rpsan\iCloudDrive\projects\hugginface_translate_txt\data\Kazimierz Dąbrowski_ William TillierPositive DisintegratioMaurice Bassett.txt"
input_file_utf8 =  r"C:\Users\rpsan\iCloudDrive\projects\hugginface_translate_txt\data\Kazimierz Dąbrowski_ William TillierPositive DisintegratioMaurice Bassett_utf8.txt"

with open(input_file_utf8, 'r', encoding="utf-8") as file:
    book_text = file.read()

# # Translate the book text
# translated_book = translate_text(book_text)

# # Save the translated text to a new file
# with open('../output/book_translated.txt', 'w', encoding='utf-8') as file:
#     file.write(translated_book)



# Translate the book text and output sentences
translated_output,translated_text = translate_and_output_sentences(book_text)

# Save the original and translated sentences to a new file
with open('../output/book_translated_sentences.txt_batch16_utf8_BERT', 'w', encoding='utf-8') as file:
    for item in translated_output:
        file.write(item)
with open('../output/book_translated_sentences.txt_batch16_utf8_only_portuguese_BERT', 'w', encoding='utf-8') as file:
        file.write(translated_text)

Translating: 100%|██████████| 95/95 [05:18<00:00,  3.36s/it]
